Curso ministrado por Guilherme Silveira

** Machine Learning:** *validação de modelos*

## Setup do Curso

### Importando as Bibliotecas

In [103]:
import pandas as pd

Importando os Dados

In [104]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


# Machine Learning Models

## Análises Iniciais

In [105]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [106]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified', random_state=SEED)
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 52.72%


In [107]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


Alterando o Seed:

In [108]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 0
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


dummy_stratified = DummyClassifier(strategy='stratified', random_state=SEED)
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100
print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)


modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi de 50.04%
A acurácia foi 75.48%


O modelo apresenta um valor elevado da *acurácia*. <p>
Entretanto, o valor está dependendo do valor do SEED, que representa uma aleatoriedade... <p>
E não é legal ter o modelo dependendo da aleatoriedade.

### Validação Cruzada

In [109]:
from sklearn.model_selection import cross_validate

modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
resultados['test_score']
print('Os resultados das três separações foram:', resultados['test_score'])

media = resultados['test_score'].mean()
desvio = resultados['test_score'].std()
print('Acurácia está entre %.2f%% e %.2f%%' % ((media - 2 * desvio)*100 , (media + 2 * desvio)*100))

Os resultados das três separações foram: [0.75704859 0.7629763  0.75337534]
Acurácia está entre 74.99% e 76.57%


Alguns *papers* indicam que o *cross validation* ideal seria entre 5 e 10. <p>
O *Cross Validation* não depende da aleatoriedade, sendo um modelo determinístico.

### Kfold com aleatorização

In [110]:
def imprime_resultados(resultados):
    media = resultados['test_score'].mean()
    desvio = resultados['test_score'].std()
    print('Média é de %0.2f%%' % (media*100))
    print('Acurácia está entre %.2f%% e %.2f%%' % ((media - 2 * desvio)*100 , (media + 2 * desvio)*100))

In [111]:
from sklearn.model_selection import KFold

SEED = 0
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
resultados['test_score']

print('Sem o Shuffle:')
imprime_resultados(resultados)

Sem o Shuffle:
Média é de 75.78%
Acurácia está entre 74.37% e 77.19%


In [112]:
from sklearn.model_selection import KFold

SEED = 0
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
resultados['test_score']

print('Com o Shuffle:')
imprime_resultados(resultados)

Com o Shuffle:
Média é de 75.78%
Acurácia está entre 73.79% e 77.77%


### Stratificação do KFold

Utilizado quando existe o desbalanço entre as classes, sendo mais indicado do que o *KFold*

In [113]:
from sklearn.model_selection import StratifiedKFold

SEED = 0
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
resultados['test_score']

print('Com o Shuffle:')
imprime_resultados(resultados)

Com o Shuffle:
Média é de 75.78%
Acurácia está entre 73.35% e 78.21%


### Aleatoriedade com Agrupamento

Gerando dados aleatórios para avaliação do modelo para a validação do estimador

In [114]:
np.random.seed(SEED)
dados['modelo'] = dados['idade_do_modelo'] + np.random.randint(-2, 3, size=10000)
dados['modelo'] = dados['modelo'] + abs(dados['modelo'].min()) +1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,22
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,15
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,7


In [115]:
dados['modelo'].value_counts()

20    836
19    797
21    762
18    734
16    717
17    709
15    628
14    602
22    573
13    543
12    463
11    424
23    394
10    394
9     315
8     280
7     224
24    183
6     178
5      97
4      82
3      37
2      20
1       8
Name: modelo, dtype: int64

In [122]:
from sklearn.model_selection import GroupKFold

SEED = 0
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, groups = dados['modelo'], cv = cv, return_train_score=False)
resultados['test_score']

print('Com o Shuffle:')
imprime_resultados(resultados)

Com o Shuffle:
Média é de 75.80%
Acurácia está entre 71.18% e 80.41%


### Cross Validation com *StandardScaler*

In [129]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 0
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)


modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)
acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 76.64%


In [130]:
SEED = 0
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
resultados = cross_validate(modelo, x, y, groups = dados['modelo'], cv = cv, return_train_score=False)
resultados['test_score']

print('Utilizando o ')
imprime_resultados(resultados)

Com o Shuffle:
Média é de 77.25%
Acurácia está entre 72.87% e 81.64%


In [132]:
from sklearn.pipeline import Pipeline

SEED = 0
np.random.seed(SEED)


scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('Transformação', scaler), ('Estimador', modelo)])
pipeline


cv = GroupKFold(n_splits=10)
resultados = cross_validate(pipeline, x, y, groups = dados['modelo'], cv = cv, return_train_score=False)
resultados['test_score']

imprime_resultados(resultados)

Média é de 76.77%
Acurácia está entre 73.06% e 80.48%
